In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras import activations
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, AveragePooling2D, Flatten, Dense 

In [2]:
data,info = tfds.load(
    'colorectal_histology',
    split=['train'],
    shuffle_files=True,
    batch_size=-1,
    as_supervised=True,
    with_info=True)

In [4]:
info

tfds.core.DatasetInfo(
    name='colorectal_histology',
    full_name='colorectal_histology/2.0.0',
    description="""
    Classification of textures in colorectal cancer histology. Each example is a 150 x 150 x 3 RGB image of one of 8 classes.
    """,
    homepage='https://zenodo.org/record/53169#.XGZemKwzbmG',
    data_path='C:\\Users\\justf\\tensorflow_datasets\\colorectal_histology\\2.0.0',
    file_format=tfrecord,
    download_size=246.14 MiB,
    dataset_size=179.23 MiB,
    features=FeaturesDict({
        'filename': Text(shape=(), dtype=string),
        'image': Image(shape=(150, 150, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=8),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=5000, num_shards=2>,
    },
    citation="""@article{kather2016multi,
      title={Multi-class texture analysis in colorectal cancer histology},
      author={Kather, Jakob Nikolas 

In [3]:
images,labels = tfds.as_numpy(data[0])

In [4]:
def dataset_maker(img_data,labels):
    images = []
    for img in img_data:
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,dsize=(224,224),interpolation=cv2.INTER_AREA)
        img = img.astype('float32')
        img /= 255
        images.append(img)
    images = np.array(images)
    labels = labels.astype('uint32')
    return images,labels

In [5]:
images,labels = dataset_maker(images,labels)

In [6]:
train_ds = tf.data.Dataset.from_tensor_slices((images[:4000],labels[:4000])).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((images[4000:],labels[4000:])).batch(32)

## GoogleNet from scratch

In [22]:
class InceptionModule(tf.keras.layers.Layer):
    def __init__(self, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj):
        super(InceptionModule, self).__init__()
        
        self.conv1 = Conv2D(filters_1x1, kernel_size=(1, 1), padding='same', activation='relu')
        
        self.conv2_reduce = Conv2D(filters_3x3_reduce, kernel_size=(1, 1), padding='same', activation='relu')
        self.conv2 = Conv2D(filters_3x3, kernel_size=(3, 3), padding='same', activation='relu')
        
        self.conv3_reduce = Conv2D(filters_5x5_reduce, kernel_size=(1, 1), padding='same', activation='relu')
        self.conv3 = Conv2D(filters_5x5, kernel_size=(5, 5), padding='same', activation='relu')
        
        self.pool_proj = Conv2D(filters_pool_proj, kernel_size=(1, 1), padding='same', activation='relu')
        
    def call(self, inputs):
        conv1_out = self.conv1(inputs)
        
        conv2_reduce_out = self.conv2_reduce(inputs)
        conv2_out = self.conv2(conv2_reduce_out)
        
        conv3_reduce_out = self.conv3_reduce(inputs)
        conv3_out = self.conv3(conv3_reduce_out)
        
        pool_out = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(inputs)
        pool_proj_out = self.pool_proj(pool_out)
        
        inception_out = concatenate([conv1_out, conv2_out, conv3_out, pool_proj_out], axis=-1)
        
        return inception_out
    
class GoogleNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(GoogleNet, self).__init__()
        
        self.conv1 = Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same', activation='relu')
        self.max_pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')
        
        self.conv2_reduce = Conv2D(64, kernel_size=(1, 1), padding='same', activation='relu')
        self.conv2 = Conv2D(192, kernel_size=(3, 3), padding='same', activation='relu')
        self.max_pool2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')
        
        self.inception3a = InceptionModule(64, 96, 128, 16, 32, 32)
        self.inception3b = InceptionModule(128, 128, 192, 32, 96, 64)
        self.max_pool3 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')
        
        self.inception4a = InceptionModule(192, 96, 208, 16, 48, 64)
        self.inception4b = InceptionModule(160, 112, 224, 24, 64, 64)
        self.inception4c = InceptionModule(128, 128, 256, 24, 64, 64)
        self.inception4d = InceptionModule(112, 144, 288, 32, 64, 64)
        self.inception4e = InceptionModule(256, 160, 320, 32, 128, 128)
        self.max_pool4 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')
        
        self.inception5a = InceptionModule(256, 160, 320, 32, 128, 128)
        self.inception5b = InceptionModule(384, 192, 384, 48, 128, 128)
        
        self.avg_pool = AveragePooling2D(pool_size=(7, 7), strides=(1, 1), padding='valid')
        self.dropout = Dropout(0.4)
        self.fc1 = Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.max_pool1(x)
        
        x = self.conv2_reduce(x)
        x = self.conv2(x)
        x = self.max_pool2(x)
        
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.max_pool3(x)
        
        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.max_pool4(x)
        
        x = self.inception5a(x)
        x = self.inception5b(x)

        x = self.avg_pool(x)
        x = self.dropout(x)
        
        return self.fc1(x)


In [23]:
model = GoogleNet(num_classes=8)

In [24]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [25]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [26]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [27]:
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [28]:
EPOCHS = 5
with tf.device('/device:GPU:0'):
    for epoch in range(EPOCHS):
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

        for images, labels in train_ds:
            train_step(images, labels)
        for test_images, test_labels in test_ds:
            test_step(test_images, test_labels)
        print(
            f'Epoch {epoch + 1}, '
            f'Loss: {train_loss.result()}, '
            f'Accuracy: {train_accuracy.result() * 100}, '
            f'Test Loss: {test_loss.result()}, '
            f'Test Accuracy: {test_accuracy.result() * 100}'
        )

Epoch 1, Loss: 1.853042483329773, Accuracy: 12.642969131469727, Test Loss: 1.7231708765029907, Test Accuracy: 12.974722862243652
Epoch 2, Loss: 1.4905524253845215, Accuracy: 13.198436737060547, Test Loss: 1.4757370948791504, Test Accuracy: 13.60035228729248
Epoch 3, Loss: 1.3746355772018433, Accuracy: 13.424219131469727, Test Loss: 1.326932430267334, Test Accuracy: 13.60035228729248
Epoch 4, Loss: 1.371283769607544, Accuracy: 13.34218692779541, Test Loss: 1.368717908859253, Test Accuracy: 13.628646850585938
Epoch 5, Loss: 1.2902132272720337, Accuracy: 13.383594512939453, Test Loss: 1.2748913764953613, Test Accuracy: 13.616071701049805


In [29]:
model.summary()

Model: "google_net_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_114 (Conv2D)         multiple                  9472      
                                                                 
 max_pooling2d_8 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 conv2d_115 (Conv2D)         multiple                  4160      
                                                                 
 conv2d_116 (Conv2D)         multiple                  110784    
                                                                 
 max_pooling2d_9 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 inception_module_18 (Incept  multiple                

In [30]:
model.save('scratch/')

INFO:tensorflow:Assets written to: scratch/assets


INFO:tensorflow:Assets written to: scratch/assets


## Transfer learning with Inception

In [31]:
inception = keras.applications.InceptionV3(input_shape=(224,224,3),include_top=False)

87910968/87910968 [==============================] - 19s 0us/step


In [32]:
inception.trainable=False

In [33]:
inception.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_171 (Conv2D)            (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d_171[0][0]']             
 alization)                     )                                                      

In [35]:
with tf.device('/device:CPU:0'):
    in_model = Sequential([
        inception,
        keras.layers.Flatten(),
        keras.layers.Dense(4096,activation=activations.relu),
        keras.layers.Dense(2048,activation=activations.relu),
        keras.layers.Dense(1000,activation=activations.relu),
        keras.layers.Dense(100,activation=activations.relu),
        keras.layers.Dense(8,activation=activations.softmax)])

In [36]:
in_model.compile(optimizer=keras.optimizers.Adam(),
                 loss=keras.losses.SparseCategoricalCrossentropy(),
                 metrics=keras.metrics.SparseCategoricalAccuracy())

In [38]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2),
    tf.keras.callbacks.ModelCheckpoint(filepath='saved_model/model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]
with tf.device('/device:CPU:0'):
    in_model.fit(
        train_ds,
        epochs=5,
        callbacks=my_callbacks,
        verbose = 1,
        validation_data = test_ds)

Epoch 1/5
125/125 [==============================] - 289s 2s/step - loss: 5.2830 - sparse_categorical_accuracy: 0.6165 - val_loss: 0.5192 - val_sparse_categorical_accuracy: 0.8300
Epoch 2/5
125/125 [==============================] - 229s 2s/step - loss: 0.4006 - sparse_categorical_accuracy: 0.8670 - val_loss: 0.5157 - val_sparse_categorical_accuracy: 0.8630
Epoch 3/5
125/125 [==============================] - 219s 2s/step - loss: 0.3050 - sparse_categorical_accuracy: 0.9015 - val_loss: 0.8312 - val_sparse_categorical_accuracy: 0.8130
Epoch 4/5
125/125 [==============================] - 213s 2s/step - loss: 0.2222 - sparse_categorical_accuracy: 0.9215 - val_loss: 0.5319 - val_sparse_categorical_accuracy: 0.8690


In [40]:
with tf.device('/device:CPU:0'):
    in_model.evaluate(test_ds)

32/32 [==============================] - 27s 830ms/step - loss: 0.5319 - sparse_categorical_accuracy: 0.8690
